## Degrees ##

In [ ]:
import csv
import sys

from util import Node, StackFrontier, QueueFrontier


In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/small.zip', 'r') as ZipObj:
  ZipObj.extractall()

In [ ]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


In [ ]:
## To be implemented
def shortest_path( source, target ):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """

    solution = list()
    explored = set()

    solution_found = False
    empty = False

    start = Node(state=source, parent=None, action=None)
    frontier = QueueFrontier()
    frontier.add(start)

    while not solution_found:
        if frontier.empty():
            solution_found = True
            empty = True
        
        # Choose a node from frontier
        node = frontier.remove()

        # If node is the target, then we have a solution
        if node.state == target:
            solution_found = True
            while node.parent is not None:
                pid, mid = node.state, node.action
                solution.append((mid, pid))
                node = node.parent
            solution.reverse()

        # Mark node as explored
        explored.add(node)
        neighbors = neighbors_for_person(node.state)

        for neighbor in neighbors:
            child = Node(state=neighbor[1], action=neighbor[0], parent=node)
            # Add neighbor to frontier
            frontier.add(child)

            # If any child node from neighbors is the target, then we have a solution
            if child.state == target:
                solution_found = True
                while child.parent is not None:
                    pid, mid = child.state, child.action
                    solution.append((mid, pid))
                    child = child.parent
                solution.reverse()

    if solution_found:
        if empty:
            return None
        return solution
    # TODO
    raise NotImplementedError


In [ ]:
## This cell works as the main - start working with the small folder and when you are done comment it and 
##comment out the line for the large folder

##directory ="./small"
directory="./large"

# Load data from files into memory
print("Loading data...")
load_data(directory)
print("Data loaded.")



Loading data...
Data loaded.


In [ ]:
source = person_id_for_name(input("Name: "))
if source is None:
    sys.exit("Person not found.")
target = person_id_for_name(input("Name: "))
if target is None:
    sys.exit("Person not found.")

path = shortest_path(source, target)

if path is None:
    print("Not connected.")
else:
    degrees = len(path)
    print(f"{degrees} degrees of separation.")
    path = [(None, source)] + path
    for i in range(degrees):
        person1 = people[path[i][1]]["name"]
        person2 = people[path[i + 1][1]]["name"]
        movie = movies[path[i + 1][0]]["title"]
        print(f"{i + 1}: {person1} and {person2} starred in {movie}")


Name: Frank Sinatra
Name: Elton John
3 degrees of separation.
1: Frank Sinatra and George Kennedy starred in Dirty Dingus Magee
2: George Kennedy and Oliver Reed starred in Hired to Kill
3: Oliver Reed and Elton John starred in Tommy
